In [4]:
# dependencies
import pandas as pd

from bs4 import BeautifulSoup
from IPython.display import Image
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
# import Splinter and set the chromedriver path
from splinter import Browser
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/pbernhardt/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


Step One: the scraping

In [3]:
# scraping NASA Mars News
# collect the latest News Title and Paragraph Text from redplanetscience.com...
url = "https://redplanetscience.com"
browser.visit(url)
print(url)

https://redplanetscience.com


In [ ]:
# ...and save them to variables for use later.

# <div class="content_title">NASA's Mars Reconnaissance Orbiter Undergoes Memory Update</div>
# div.content_title seems to be the latest news title.

# <div class="article_teaser_body">...</div>
# div.article_teaser_body seems to be the paragraph text from the latest news title.


In [ ]:
# scraping JPL Mars Space Images - Featured Image
# visit spaceimages-mars.com...
domain = "https://spaceimages-mars.com/"
print(domain)

In [ ]:
# ...and use Splinter to navigate the site & find the URL of the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.
# Find the URL of the *full size* .jpg image.
# Save a complete url string for this image, e.g.
# featured_image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

# The current featured image seems to be
# <img class="headerimage fade-in" src="image/featured/mars3.jpg">
# featured_image_relative_url = somehow get what's in src out of the above
featured_image_relative_url = "image/featured/mars3.jpg"
featured_image_url = f"{domain}{featured_image_relative_url}"
print(featured_image_url)
Image(featured_image_url)


In [ ]:
xpath = '//td//a[@class="list_image"]/img'

# Use splinter to Click the "Mars in natural color in 2007" image 
# to bring up the full resolution image
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

# Scrape the browser into soup and use soup to find the full resolution image of mars
# Save the image url to a variable called `img_url`
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
img_url = soup.find("img", class_="jpg")["src"]
img_url
Image(url='img.png')

In [17]:
# scraping Mars Facts
# visit galaxyfacts-mars.com
url = "https://galaxyfacts-mars.com"
browser.visit(url)


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=63829): Max retries exceeded with url: /session/223e8e9abbe81ac914f21881b46cb881/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fadb24f9278>: Failed to establish a new connection: [Errno 61] Connection refused',))

In [28]:
# and use Pandas to scrape the table containing facts about the planet including:
# diameter, mass, usw.

# <table class="table table-striped"> seems to be this table.
HTML_tables = pd.read_html(url)
# HTML_tables
# type(HTML_tables)
df_from_html_table = HTML_tables[0]
df_from_html_table

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [33]:
# Use Pandas to convert the data to a HTML table string.
# Something about scraping the Wikipedia entry for state capitols.
# url = 'https://en.wikipedia.org/wiki/List_of_capitals_in_the_United_States'
html_from_df = df_from_html_table.to_html()
print(html_from_df)


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Distance from Sun:</td>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Length of Year:</td>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Temperature:</td>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody

In [ ]:
# scraping Mars Hemispheres
# Visit marshemispheres.com...
print("https://marshemispheres.com")

In [ ]:
# ...to obtain high resolution images for each of Mars's hemispheres.
# Click each of the links to the hemispheres to find the URL of the full resolution image.
# Save both the image url string for the full resolution hemisphere image,
# and the Hemisphere title containing the hemisphere name.
# Use a Python dictionary to store the data using the keys img_url and title.
# Append the dictionary with the image url string and the hemisphere title to a list.
# This list will contain one dictionary for each hemisphere.

# Example:
# hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#    {"title": "Cerberus Hemisphere", "img_url": "..."},
#    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]

hemisphere_image_urls = []
hemisphere_image_urls

In [9]:
# close the session
browser.quit()


Step 2: MongoDB and Flask

In [ ]:
# Use MongoDB with Flask templating to create a new HTML page
# that displays all of the information that was scraped from the URLs above.

# Start by converting your Jupyter notebook into a Python script called scrape_mars.py
# with a function called scrape that will execute all of your scraping code from above
# and return one Python dictionary containing all of the scraped data.

# Next, create a route called /scrape that will import your scrape_mars.py script and
# call your scrape function.

# Store the return value in Mongo as a Python dictionary.

# Create a root route / that will query your Mongo database and
# pass the mars data into an HTML template to display the data.

# Create a template HTML file called index.html that will take the mars data dictionary and
# display all of the data in the appropriate HTML elements.
# Use the following as a guide for what the final product should look like,
# but feel free to create your own design:
print("https://umn.bootcampcontent.com/University-of-Minnesota-Boot-Camp/uofm-stp-data-pt-12-2020-u-c/tree/master/02-Homework/12-Web-Scraping-and-Document-Databases/Instructions"
)

Step 3: Submit

In [ ]:
# Upload the Jupyter Notebook containing the scraping code and
# screenshots of your final application
# to your GitHub repository and submit the link.
# Ensure your repository has regular commits (i.e. 20+ commits) and
# a thorough README.md file.

Hints

In [ ]:
# Use Splinter to navigate the sites when needed and
# BeautifulSoup to help find and parse out the necessary data.

# Use Pymongo for CRUD applications for your database.
# For this homework, you can simply overwrite the existing document each time
# the /scrape url is visited and new data is obtained.

# Use Bootstrap to structure your HTML template.